```
from: [[1,2, ENTITYNAME, [3,4, ENTITYNAME]], [123,135, ANOTHERENT, ...], ...]
+ a source conllu file

-> conllu with new mentions
```

In [1]:
from typing import List, Tuple
import udapi
import udapi.block.corefud.movehead
import udapi.block.corefud.removemisc

MentionWithEid = List[List[Tuple[int, int, int]]]
MentionOnly = List[List[Tuple[int, int]]]

def wipe_conllu(path, output_path):
    doc = udapi.block.read.conllu.Conllu(files=[path]).read_documents()[0]
    udapi.block.corefud.removemisc.RemoveMisc(
        attrnames="Entity,SplitAnte,Bridge").apply_on_document(doc)
    udapi.block.write.conllu.Conllu(files=[output_path]).apply_on_document(doc)
    
def create_conllu(path, mentions):
    doc = udapi.block.read.conllu.Conllu(files=[path]).read_documents()[0]
    udapi.block.corefud.removemisc.RemoveMisc(
        attrnames="Entity,SplitAnte,Bridge").apply_on_document(doc)
    entities = {}
    for i, tree in enumerate(doc.trees):
        print(i, tree, tree.descendants_and_empty)
        nodes = tree.descendants_and_empty
        for start, end, eid in mentions[i]:
            print(start, end, eid)
            if not eid in entities:
                entities[eid] = udapi.core.coref.CorefEntity(f"c{eid}")
            udapi.core.coref.CorefMention(nodes[start : end + 1], entity=entities[eid])
    doc._eid_to_entity = {entity._eid: entity for entity in sorted(entities.values())}
    udapi.block.corefud.movehead.MoveHead(bugs="ignore").apply_on_document(doc)
    udapi.block.write.conllu.Conllu(files=[path]).apply_on_document(doc)

    for mention in doc.coref_mentions:
        mention.words = [mention.head]

# mentions = [[(0,5), (6,7)], [(0,1), (2,3), (4,5)], [(13, 18), (24, 30)]]
mentions = [[], [], []]
path = "tmp.conllu"
create_conllu(path, mentions)



0 <016056, <ROOT>> [Node<016056#1, Pinefull>, Node<016056#2, død>, Node<016056#3, for>, Node<016056#4, den>, Node<016056#5, irske>, Node<016056#6, pub>, Node<016056#7, ?>]
1 <016057, <ROOT>> [Node<016057#1, Irene>, Node<016057#2, diskuterer>, Node<016057#3, over>, Node<016057#4, mørkebrunt>, Node<016057#5, øl>, Node<016057#6, .>]
2 <016058, <ROOT>> [Node<016058#1, Micheál>, Node<016058#2, Martin>, Node<016058#3, er>, Node<016058#4, på>, Node<016058#5, alles>, Node<016058#6, lepper>, Node<016058#7, .>]


In [4]:
paths = [
    "../data/CRAC/CorefUD_Czech-PCEDT/cs_pcedt-corefud-dev.conllu",
    "../data/CRAC/CorefUD_Czech-PDT/cs_pdt-corefud-dev.conllu",
    "../data/CRAC/CorefUD_Lithuanian-LCC/lt_lcc-corefud-dev.conllu",
    "../data/CRAC/CorefUD_Polish-PCC/pl_pcc-corefud-dev.conllu",
    "../data/CRAC/CorefUD_Russian-RuCor/ru_rucor-corefud-dev.conllu"
]
wiped_folder = "models/corpiper/submissions"
wiped_paths = [p.split("/")[-1] for p in paths]
wiped_paths = [f"{wiped_folder}/{p}" for p in wiped_paths]
for p, wp in zip(paths, wiped_paths):
    wipe_conllu(p, wp)

In [6]:
# zip the submissions folder 
!zip -r models/corpiper/submissions_test.zip models/corpiper/submissions

  adding: models/corpiper/submissions/ (stored 0%)
  adding: models/corpiper/submissions/en_gum-corefud-dev.conllu (deflated 80%)
  adding: models/corpiper/submissions/cs_pcedt-corefud-dev.conllu (deflated 83%)
  adding: models/corpiper/submissions/de_potsdamcc-corefud-dev.conllu (deflated 82%)
  adding: models/corpiper/submissions/no_bokmaalnarc-corefud-dev.conllu (deflated 81%)
  adding: models/corpiper/submissions/de_parcorfull-corefud-dev.conllu (deflated 82%)
  adding: models/corpiper/submissions/lt_lcc-corefud-dev.conllu (deflated 85%)
  adding: models/corpiper/submissions/es_ancora-corefud-dev.conllu (deflated 82%)
  adding: models/corpiper/submissions/cs_pdt-corefud-dev.conllu (deflated 83%)
  adding: models/corpiper/submissions/hu_korkor-corefud-dev.conllu (deflated 82%)
  adding: models/corpiper/submissions/pl_pcc-corefud-dev.conllu (deflated 84%)
  adding: models/corpiper/submissions/tr_itcc-corefud-dev.conllu (deflated 85%)
  adding: models/corpiper/submissions/en_parcorful